In [1]:
import tensorflow as tf
tf.test.is_gpu_available()

W1210 14:06:53.895703 139990078469888 __init__.py:321] Limited tf.compat.v2.summary API due to missing TensorBoard installation.
W1210 14:06:54.049607 139990078469888 __init__.py:321] Limited tf.compat.v2.summary API due to missing TensorBoard installation.
W1210 14:06:54.137888 139990078469888 __init__.py:352] Limited tf.summary API due to missing TensorBoard installation.


False

In [2]:
%load_ext autoreload
%autoreload 2

W1210 14:06:54.458840 139990078469888 __init__.py:321] Limited tf.compat.v2.summary API due to missing TensorBoard installation.


In [3]:
%matplotlib inline
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from collections import Counter
import itertools
import json
import sys
# sys.path.append("..")
from utils import data_proc_tools as dpt
from utils import plot_tools as pt
from utils.custom_metrics import recall, precision, binary_accuracy
from utils.custom_metrics import recall_np, precision_np, binary_accuracy_np, multilabel_confusion_matrix
import random
random.seed(42)
random_state=1000
pd.set_option('display.max_colwidth', -1)
import pylab

pylab.rcParams['figure.figsize'] = (8.0, 10.0)

Using TensorFlow backend.


In [4]:
dir = '/vol/medic02/users/ag6516/radiology_report_summarisation/'
data_dir = dir + 'data/'

aug = 'aug'

model_output_dir = dir + 'trained_models/cnn2hierseq_att/'

train_df = pd.read_pickle(data_dir + 'train/{}_train.pkl'.format(aug))
val_df = pd.read_pickle(data_dir + 'val/val.pkl')

## Load and prepare sequence data for training

In [5]:
train_df.head()

,examid,report,all_mesh,single_mesh
0,CXR1000_IM-0003,"[increased, opacity, within, right, upper, lobe, possible, mass, associated, area, atelectasis, focal, consolidation, ., cardiac, silhouette, within, normal, limits, ., opacity, left, midlung, overlying, posterior, left, 5th, rib, may, represent, focal, airspace, disease, ., increased, opacity, right, upper, lobe, associated, atelectasis, may, represent, focal, consolidation, mass, lesion, atelectasis, ., recommend, chest, ct, evaluation, ., opacity, overlying, left, 5th, rib, may, represent, focal, airspace, disease]","[opacity, lung, lingula, opacity, lung, upper_lobe, right, pulmonary_atelectasis, upper_lobe, right]","[opacity, lung, upper_lobe, right]"
1,CXR1001_IM-0004,"[interstitial, markings, diffusely, prominent, throughout, lungs, ., heart, size, normal, ., pulmonary, normal, ., diffuse, fibrosis]","[diffuse, markings, lung, bilateral, interstitial, diffuse, prominent]","[markings, lung, bilateral, interstitial, diffuse, prominent]"
2,CXR1002_IM-0004,"[status, post, left, mastectomy, ., heart, size, normal, ., lungs, clear]",[left],[left]
3,CXR1003_IM-0005,"[heart, size, pulmonary, vascularity, appear, within, normal, limits, ., retrocardiac, soft, tissue, density, present, ., appears, air, within, suggest, represents, hiatal, hernia, ., vascular, calcification, noted, ., calcified, granuloma, seen, ., interval, development, bandlike, opacity, left, lung, base, ., may, represent, atelectasis, ., osteopenia, present, spine, ., retrocardiac, soft, tissue, density, ., appearance, suggests, hiatal, hernia, ., left, base, bandlike, opacity, ., appearance, suggests, atelectasis]","[bone_diseases_metabolic, spine, calcified_granuloma, calcinosis, blood_vessels, density, retrocardiac, opacity, lung, base, left]","[opacity, lung, base, left]"
4,CXR1004_IM-0005,"[heart, ,, pulmonary, mediastinum, within, normal, limits, ., aorta, tortuous, ectatic, ., degenerative, changes, acromioclavicular, joints, ., degenerative, changes, spine, ., ivc, identified]","[aorta, tortuous, catheters_indwelling, shoulder, bilateral, degenerative, spine, degenerative]","[shoulder, bilateral, degenerative]"


In [6]:
# prepend and append start and end tokens to mesh captions and text reports
start_token = 'start'
end_token = 'end'
unknown_token = '**unknown**'
max_mesh_length = 13 # avg. + 1std. + start + end
max_report_length = 37 # avg. + 1std. + start + end

train_df['pad_mesh_caption'] = train_df.all_mesh.apply(lambda x: dpt.pad_sequence(x, max_mesh_length, start_token, end_token))
train_df['pad_text_report'] = train_df.report.apply(lambda x: dpt.pad_sequence(x, max_report_length, start_token, end_token))

val_df['pad_mesh_caption'] = val_df.all_mesh.apply(lambda x: dpt.pad_sequence(x, max_mesh_length, start_token, end_token))
val_df['pad_text_report'] = val_df.report.apply(lambda x: dpt.pad_sequence(x, max_report_length, start_token, end_token))

## Vectorise text reports and mesh captions

In [7]:
train_mesh = list(train_df.pad_mesh_caption)
train_reports = list(train_df.pad_text_report)

# vectorize mesh captions
dpt.mesh_to_vectors(train_mesh, dicts_dir=data_dir+'dicts/', 
                    load_dicts=True, save=True, output_dir=data_dir+'train/')

# vectorise reports
dpt.reports_to_vectors(train_reports, dicts_dir=data_dir+'dicts/', 
                       load_dicts=True, save=True, output_dir=data_dir+'train/')

In [8]:
word_to_id, id_to_word = dpt.load_report_dicts(data_dir+'dicts/')
mesh_to_id, id_to_mesh = dpt.load_mesh_dicts(data_dir+'dicts/')

report_vocab_length = len(word_to_id)
mesh_vocab_length = len(mesh_to_id)

In [9]:
report_vocab_length, mesh_vocab_length

(1475, 128)

In [10]:
# Create arrays of indixes for input sentences, output entities and shifted output entities (t-1)
train_token_ids_array = np.load(data_dir + 'train/token_ids_array.npy')
train_mesh_ids_array = np.load(data_dir + 'train/mesh_ids_array.npy')
train_mesh_ids_array_shifted =[np.concatenate((mesh_to_id[start_token], t[:-1]), axis=None) for t in train_mesh_ids_array]
train_mesh_ids_array_shifted = np.asarray(train_mesh_ids_array_shifted)

val_token_ids_array = np.load(data_dir + 'val/token_ids_array.npy')
val_mesh_ids_array = np.load(data_dir + 'val/mesh_ids_array.npy')
val_mesh_ids_array_shifted = [np.concatenate((mesh_to_id[start_token], t[:-1]), axis=None) for t in val_mesh_ids_array]
val_mesh_ids_array_shifted = np.asarray(val_mesh_ids_array_shifted)

In [11]:
# one-hot-encode
one_hot_reports_train = dpt.one_hot_sequence(train_token_ids_array, report_vocab_length)
one_hot_mesh_train = dpt.one_hot_sequence(train_mesh_ids_array, mesh_vocab_length)
one_hot_mesh_shifted_train = dpt.one_hot_sequence(train_mesh_ids_array_shifted, mesh_vocab_length)

one_hot_reports_val = dpt.one_hot_sequence(val_token_ids_array, report_vocab_length)
one_hot_mesh_val = dpt.one_hot_sequence(val_mesh_ids_array, mesh_vocab_length)
one_hot_mesh_shifted_val = dpt.one_hot_sequence(val_mesh_ids_array_shifted, mesh_vocab_length)

In [12]:
train_token_ids_array.shape, one_hot_mesh_train.shape, one_hot_mesh_shifted_train.shape

((5148, 37), (5148, 13, 128), (5148, 13, 128))

## Train CNN-to-Seq Model

In [37]:
from utils.custom_metrics import recall, precision, binary_accuracy
from utils.cnn_textsum_models import HierCNN2SeqAtt

input_dim = len(word_to_id)
output_dim = len(mesh_to_id)
embedding_dim = 256
word_conv_dim = 64
phrase_conv_dim = 128
conv1_kernel = 1
conv2_kernel = 3
conv3_kernel = 5
input_seq_length = max_report_length
output_seq_length = max_mesh_length
epochs = 70
optimizer = 'adam'
loss='categorical_crossentropy'
batch_size = 128

new_experiment = HierCNN2SeqAtt(epochs=epochs,
                               metrics=['accuracy', binary_accuracy,recall,precision],
                               optimizer=optimizer,
                               loss=loss,
                               batch_size=batch_size, 
                               input_dim=input_dim,
                               output_dim=output_dim,
                               embedding_dim=embedding_dim,
                               word_conv_dim=word_conv_dim,
                               phrase_conv_dim=phrase_conv_dim,
                               conv1_kernel=conv1_kernel,
                               conv2_kernel=conv2_kernel,
                               conv3_kernel=conv3_kernel,
                               input_seq_length=input_seq_length,
                               output_seq_length=output_seq_length,
                               verbose=True)

new_experiment.build_model()
new_experiment.model.summary()

Word encoder emb:  (None, 37, 1475)
Word conv1 output:  (None, 37, 64)
Word conv2 output:  (None, 37, 64)
Word conv3 output:  (None, 37, 64)
Word Concat:  (None, 37, 192)
Max pool outputs:  (None, 37, 192)
Phrase Concat:  (None, 37, 640)
Encoder states:  (None, 640)
Decoder outputs:  (None, 13, 640)
Attention outputs:  (None, 13, 640)
Model: "model_17"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_29 (InputLayer)           [(None, 37, 1475)]   0                                            
__________________________________________________________________________________________________
conv1d_24 (Conv1D)              (None, 37, 64)       94464       input_29[0][0]                   
__________________________________________________________________________________________________
conv1d_25 (Conv1D)              (None, 37, 64)     

In [38]:
new_experiment.run_experiment(one_hot_reports_train, one_hot_mesh_shifted_train, one_hot_mesh_train, 
                              one_hot_reports_val, one_hot_mesh_shifted_val, one_hot_mesh_val)

Word encoder emb:  (None, 37, 1475)
Word conv1 output:  (None, 37, 64)
Word conv2 output:  (None, 37, 64)
Word conv3 output:  (None, 37, 64)
Word Concat:  (None, 37, 192)
Max pool outputs:  (None, 37, 192)
Phrase Concat:  (None, 37, 640)
Encoder states:  (None, 640)
Decoder outputs:  (None, 13, 640)
Attention outputs:  (None, 13, 640)
Train on 5148 samples, validate on 300 samples
Epoch 1/70
5148/5148 [==============================] - 72s 14ms/sample - loss: 2.5433 - accuracy: 0.5538 - binary_accuracy: 0.9947 - recall: 0.4147 - precision: 0.7289 - val_loss: 1.5692 - val_accuracy: 0.6921 - val_binary_accuracy: 0.9965 - val_recall: 0.6066 - val_precision: 0.9202
Epoch 2/70
5148/5148 [==============================] - 60s 12ms/sample - loss: 1.6800 - accuracy: 0.6388 - binary_accuracy: 0.9963 - recall: 0.5650 - precision: 0.9447 - val_loss: 1.3122 - val_accuracy: 0.7082 - val_binary_accuracy: 0.9970 - val_recall: 0.6386 - val_precision: 0.9724
Epoch 3/70
5148/5148 [======================

5148/5148 [==============================] - 39s 8ms/sample - loss: 0.0201 - accuracy: 0.9981 - binary_accuracy: 1.0000 - recall: 0.9973 - precision: 0.9985 - val_loss: 0.7936 - val_accuracy: 0.8726 - val_binary_accuracy: 0.9982 - val_recall: 0.8654 - val_precision: 0.8995
Epoch 29/70
5148/5148 [==============================] - 40s 8ms/sample - loss: 0.0181 - accuracy: 0.9984 - binary_accuracy: 1.0000 - recall: 0.9978 - precision: 0.9988 - val_loss: 0.8306 - val_accuracy: 0.8738 - val_binary_accuracy: 0.9982 - val_recall: 0.8670 - val_precision: 0.8951
Epoch 30/70
5148/5148 [==============================] - 39s 8ms/sample - loss: 0.0409 - accuracy: 0.9918 - binary_accuracy: 0.9999 - recall: 0.9890 - precision: 0.9938 - val_loss: 0.7714 - val_accuracy: 0.8723 - val_binary_accuracy: 0.9982 - val_recall: 0.8628 - val_precision: 0.8983
Epoch 31/70
5148/5148 [==============================] - 39s 8ms/sample - loss: 0.0518 - accuracy: 0.9896 - binary_accuracy: 0.9998 - recall: 0.9855 - pre

Epoch 57/70
5148/5148 [==============================] - 40s 8ms/sample - loss: 0.0041 - accuracy: 0.9995 - binary_accuracy: 1.0000 - recall: 0.9994 - precision: 0.9996 - val_loss: 0.8270 - val_accuracy: 0.8797 - val_binary_accuracy: 0.9982 - val_recall: 0.8743 - val_precision: 0.8971
Epoch 58/70
5148/5148 [==============================] - 39s 8ms/sample - loss: 0.0037 - accuracy: 0.9995 - binary_accuracy: 1.0000 - recall: 0.9993 - precision: 0.9996 - val_loss: 0.8403 - val_accuracy: 0.8785 - val_binary_accuracy: 0.9982 - val_recall: 0.8741 - val_precision: 0.8956
Epoch 59/70
5148/5148 [==============================] - 39s 8ms/sample - loss: 0.0037 - accuracy: 0.9995 - binary_accuracy: 1.0000 - recall: 0.9994 - precision: 0.9995 - val_loss: 0.8387 - val_accuracy: 0.8779 - val_binary_accuracy: 0.9982 - val_recall: 0.8747 - val_precision: 0.8949
Epoch 60/70
5148/5148 [==============================] - 40s 8ms/sample - loss: 0.0032 - accuracy: 0.9996 - binary_accuracy: 1.0000 - recall: 

In [39]:
model_name = new_experiment.model_name
model_output_dir = dir + 'trained_models/{}/'.format(model_name)
new_experiment.save_weights_history(model_output_dir)

## Load results of specific experiment

In [40]:
model_name = 'hiercnn2seq_att'
model_output_dir = dir + 'trained_models/{}/'.format(model_name)

In [41]:
import pickle
from utils.cnn_textsum_models import HierCNN2SeqAtt

model_name = 'hiercnn2seq_att'
embedding_dim = 256
word_conv_dim = 64
phrase_conv_dim = 128
conv1_kernel = 1
conv2_kernel = 3
conv3_kernel = 5
epochs = 70

param_fn = 'param_{}_wordconvdim_{}_phraseconvdim_{}_kernels_{}{}{}_epochs_{}.pkl'.format(model_name,\
word_conv_dim, phrase_conv_dim, conv1_kernel, conv2_kernel, conv3_kernel, epochs)
params = pickle.load(open(model_output_dir + param_fn, 'rb'))

old_experiment = HierCNN2SeqAtt(**params)
old_experiment.build_model()
old_experiment.load_weights_history(model_output_dir)

Word encoder emb:  (None, 37, 1475)
Word conv1 output:  (None, 37, 64)
Word conv2 output:  (None, 37, 64)
Word conv3 output:  (None, 37, 64)
Word Concat:  (None, 37, 192)
Max pool outputs:  (None, 37, 192)
Phrase Concat:  (None, 37, 640)
Encoder states:  (None, 640)
Decoder outputs:  (None, 13, 640)
Attention outputs:  (None, 13, 640)


In [42]:
# decode a one hot encoded string
def one_hot_decode(encoded_seq):
    return [np.argmax(vector) for vector in encoded_seq]

In [43]:
def remove_cons_duplicates(seq):
    newlst = [k for k, g in itertools.groupby(seq)]
    return newlst

In [44]:
def strip_start_end(seq, start_token='start', end_token='end'):
    stripped_seq = []
    for s in seq:
        if s not in [start_token, end_token]:
            stripped_seq.append(s)
    stripped_seq = remove_cons_duplicates(stripped_seq)
    return stripped_seq

In [45]:
def predict_sequence(experiment, source, max_seq_len, id_to_mesh, start_token='start', end_token='end'):
    # encode source
    enc_outs, h, c = experiment.encoder_model.predict(source)
    enc_state = [h,c]
    dec_state = enc_state

    # start of sequence input
    in_text = [start_token]

    # integer encoder
    in_seq_ids = dpt.mesh_to_vectors([in_text], dicts_dir=data_dir+'dicts/', 
                   load_dicts=True, save=False)
    # one-hot encode
    in_seq_onehot = dpt.one_hot_sequence(in_seq_ids, mesh_vocab_length)
    in_seq_onehot = np.array(in_seq_onehot)
    in_seq_onehot = in_seq_onehot.reshape(1, 1, in_seq_onehot.shape[-1])
    target_seq = in_seq_onehot
    
    # collect predictions
    output = []
    attention_weights = []
    max_att = []
    max_att2 = []
    for t in range(max_seq_len):
        dec_out, attention, h, c  = experiment.decoder_model.predict([enc_outs] + dec_state + [target_seq])
        dec_state = [h,c]
        # store prediction
        output.append(dec_out[0,0,:])
        dec_ind = np.argmax(dec_out, axis=-1)[0, 0]

        attention_weights.append((dec_ind, attention))
        idx = np.argmax(attention, axis=-1)[0, 0]
        max_att.append(np.argmax(attention, axis=-1)[0, 0])
        attention[:,:,idx] = 0
        max_att2.append(np.argmax(attention, axis=-1)[0, 0])
        target_seq = dec_out

    predicted_mesh_ids = one_hot_decode(output)
    predicted_mesh = [id_to_mesh[idx] for idx in predicted_mesh_ids]
    
    return predicted_mesh, attention_weights, max_att, max_att2

In [46]:
for _ in range(1):
    sample = val_df.sample(1, random_state=42)

    true_mesh_caption = list(sample.all_mesh)[0]
    sample_report = list(sample.pad_text_report)[0]

    sample_report_ids = []
    for token in sample_report:
        if token in word_to_id.keys():
            sample_report_ids.append(word_to_id[token])
        else:
            sample_report_ids.append(word_to_id[unknown_token])

    sample_report_ids = np.array(sample_report_ids)
    sample_report_ids = np.array(sample_report_ids).reshape(1, len(sample_report_ids))
    one_hot_sample_report = dpt.one_hot_sequence(sample_report_ids, report_vocab_length)
    predicted_mesh, attention_weights, max_att1, max_att2 = predict_sequence(old_experiment, 
                                  one_hot_sample_report, 
                                  max_mesh_length, 
                                  id_to_mesh,
                                  start_token=start_token,
                                  end_token=end_token)
#     predicted_mesh = predict_sequence(old_experiment.inference_model, 
#                                   sample_report_ids, 
#                                   max_mesh_length, 
#                                   id_to_mesh,
#                                   start_token, 
#                                   end_token)
#     predicted_mesh_ids = one_hot_decode(prediction)
#     predicted_mesh = [id_to_mesh[idx] for idx in predicted_mesh_ids]
    
    sample_report_s = strip_start_end(sample_report)
    predicted_mesh = strip_start_end(predicted_mesh)
    print('')
    print('Processed report: ', ' '.join(sample_report_s))
    print('True mesh caption: ', ' '.join(true_mesh_caption))
    print('Predicted mesh caption: ', ' '.join(predicted_mesh))
    print('')
    
    att_words1 = [sample_report[k] for k in max_att1]
    att_words2 = [sample_report[k] for k in max_att2]

    print('Attention word inputs 1: ', att_words1)
    print('Attention word inputs 2: ', att_words2)
    

    


Processed report:  interval cabg . sternotomy appear intact . stable , mild degenerative disc disease thoracic spine . visualized bony structures otherwise unremarkable appearance . atherosclerotic calcifications thoracic aorta . clear lungs . peripheral vascular disease
True mesh caption:  atherosclerosis aorta_thoracic thoracic_vertebrae degenerative mild
Predicted mesh caption:  atherosclerosis aorta_thoracic thoracic_vertebrae degenerative mild

Attention word inputs 1:  ['calcifications', 'calcifications', 'thoracic', 'thoracic', 'disc', 'degenerative', 'lungs', 'lungs', 'disease', 'disease', 'disease', 'disease', 'disease']
Attention word inputs 2:  ['thoracic', 'thoracic', 'calcifications', 'disc', 'disease', 'disc', '.', 'clear', 'lungs', 'lungs', 'lungs', 'lungs', 'lungs']


## Evaluate BLEU scores on all trian/val/test data

In [47]:
import nltk
from nltk.translate.bleu_score import sentence_bleu
from progress.bar import Bar

def evaluate_model(model, df, report_vocab_length):
    actual, predicted = list(), list()
    bleu1, bleu2, bleu3, bleu4 = list(), list(), list(), list()
    bar = Bar('Processing', max=len(df))
    
    for _, sample in df.iterrows():
        true_mesh_caption = sample.all_mesh
        sample_report = sample.pad_text_report

        sample_report_ids = []
        for token in sample_report:
            if token in word_to_id.keys():
                sample_report_ids.append(word_to_id[token])
            else:
                sample_report_ids.append(word_to_id[unknown_token])

        sample_report_ids = np.array(sample_report_ids).reshape(1, len(sample_report_ids))
        one_hot_sample_report = dpt.one_hot_sequence(sample_report_ids, report_vocab_length)
        predicted_mesh, _, _, _ = predict_sequence(model, 
                                  one_hot_sample_report, 
                                  max_mesh_length, 
                                  id_to_mesh,
                                  start_token=start_token,
                                  end_token=end_token)

        # sample_report = strip_start_end(sample_report)
        yhat = strip_start_end(predicted_mesh)
        reference = true_mesh_caption
        
        # calculate BLEU score
        bleu1.append(sentence_bleu([reference], yhat, weights=(1.0, 0, 0, 0)))
        bleu2.append(sentence_bleu([reference], yhat, weights=(0.5, 0.5, 0, 0)))
        bleu3.append(sentence_bleu([reference], yhat, weights=(0.3, 0.3, 0.3, 0)))
        bleu4.append(sentence_bleu([reference], yhat, weights=(0.25, 0.25, 0.25, 0.25)))
    
        # store actual and predicted
        actual.append(reference)
        predicted.append(yhat)
        
        bar.next()
        
    print('BLEU1: ', np.mean(bleu1)*100)
    print('BLEU2: ', np.mean(bleu2)*100)
    print('BLEU3: ', np.mean(bleu3)*100)
    print('BLEU4: ', np.mean(bleu4)*100)
    bar.finish()
    return actual, predicted

In [ ]:
train_actual, train_predicted = evaluate_model(old_experiment, train_df.sample(2000), report_vocab_length)

In [48]:
val_actual, val_predicted = evaluate_model(old_experiment, val_df, report_vocab_length)

/vol/medic02/users/ag6516/python3env/lib/python3.6/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/vol/medic02/users/ag6516/python3env/lib/python3.6/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/vol/medic02/users/ag6516/python3env/lib/python3.6/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Con

BLEU1:  74.43612935115107
BLEU2:  33.67999060580746
BLEU3:  24.766583200228588
BLEU4:  13.831715644175018


## Evaluate ROUGE scores on all train/val/test data

In [49]:
import rouge

evaluator = rouge.Rouge(metrics=['rouge-n', 'rouge-l', 'rouge-w'],
                       max_n=4,
                       limit_length=True,
                       length_limit=100,
                       length_limit_type='words',
                       apply_avg='Avg',
                       apply_best='Best',
                       alpha=0.5, # Default F1_score
                       weight_factor=1.2,
                       stemming=True)

In [50]:
def prepare_results(p, r, f):
    return '\t{}:\t{}: {:5.2f}\t{}: {:5.2f}\t{}: {:5.2f}'.format(metric, 'P', 100.0 * p, 'R', 100.0 * r, 'F1', 100.0 * f)

In [ ]:
train_hypotheses = [' '.join(p) for p in train_predicted]
train_references = [' '.join(a) for a in train_actual]

scores = evaluator.get_scores(train_hypotheses, train_references)

for metric, results in sorted(scores.items(), key=lambda x: x[0]):
    print(prepare_results(results['p'], results['r'], results['f']))

In [51]:
val_hypotheses = [' '.join(p) for p in val_predicted]
val_references = [' '.join(a) for a in val_actual]

scores = evaluator.get_scores(val_hypotheses, val_references)

for metric, results in sorted(scores.items(), key=lambda x: x[0]):
    print(prepare_results(results['p'], results['r'], results['f']))

	rouge-1:	P: 87.11	R: 78.56	F1: 81.15
	rouge-2:	P: 43.83	R: 36.72	F1: 38.71
	rouge-3:	P: 32.26	R: 27.58	F1: 28.69
	rouge-4:	P: 20.85	R: 18.17	F1: 18.57
	rouge-l:	P: 86.67	R: 79.45	F1: 81.82
	rouge-w:	P: 82.74	R: 63.26	F1: 69.10


## Plot Convergence

In [ ]:
from utils import plot_tools as pt

pt.plot_history(old_ex)